In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk

In [1]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [7]:
# Build a quantum circuit
circuit = qk.QuantumCircuit(3)

circuit.x(1)
circuit.h(range(3))
circuit.cx(0, 1)
circuit.draw()

┌───┐          
q_0: ┤ H ├───────■──
     ├───┤┌───┐┌─┴─┐
q_1: ┤ X ├┤ H ├┤ X ├
     ├───┤└───┘└───┘
q_2: ┤ H ├──────────
     └───┘

In [11]:
me = qk.quantum_info.Pauli("XYZ")
Clifford_Permute(circuit,me)

Pauli('ZZY')